# Welcome to Colab!

In [1]:
pip install torch

In [3]:
!pip install torch torchvision torchaudio

In [5]:
pip install torch-geometric==2.4.0 torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.5.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.5.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.8 MB/s eta 0:00:00


In [6]:
pip install torch_geometric

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric

In [7]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use data.num_node_features after data is defined
        self.conv1 = GCNConv(data.num_node_features, 4)  # 1st layer: in_dim->4
        self.conv2 = GCNConv(4, dataset.num_classes)     # 2nd layer: 4->num_classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # aggregate + activate
        x = self.conv2(x, edge_index)          # second layer
        return F.log_softmax(x, dim=1)

model = SimpleGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop (simplified)
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')

/usr/local/lib/python3.12/dist-packages/torch_geometric/typing.py:63: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_scatter/_version_cuda.so
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.12/dist-packages/torch_geometric/typing.py:101: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_sparse/_version_cuda.so
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
Processing...
Done!


GCN Test Accuracy: 0.71


In [8]:
!pip install torch_geometric

import networkx as nx
import torch
from torch_geometric.data import Data

# Create a small social graph
G = nx.Graph()
edges = [(0,1), (1,2), (2,0),    # normal users community
         (3,4), (4,5), (5, 3)]    # suspicious cluster
G.add_edges_from(edges)

# One-hot node features (6 nodes, 6-dimensional)
num_nodes = 6
features = torch.eye(num_nodes, dtype=torch.float)

# Node labels: first 3 normal (0), last 3 bots (1)
labels = torch.tensor([0,0,0, 1,1,1], dtype=torch.long)

# Convert to PyG Data
edge_index = torch.tensor(list(G.edges())).t().contiguous()
# Make edges bidirectional for undirected graph
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
data = Data(x=features, edge_index=edge_index, y=labels)

# Define train/test masks (e.g., train on 4 nodes, test on 2 nodes)
data.train_mask = torch.tensor([1,1,1,1,0,0], dtype=torch.bool)
data.test_mask  = torch.tensor([0,0,0,0,1,1], dtype=torch.bool)

# New Section

In [10]:
import networkx as nx
import torch
from torch_geometric.data import Data

# Create a small social graph
G = nx.Graph()
edges = [(0,1), (1,2), (2,0),    # normal users community
         (3,4), (4,5), (5,3)]    # suspicious cluster
G.add_edges_from(edges)

# One-hot node features (6 nodes, 6-dimensional)
num_nodes = 6
features = torch.eye(num_nodes, dtype=torch.float)

# Node labels: first 3 normal (0), last 3 bots (1)
labels = torch.tensor([0,0,0, 1,1,1], dtype=torch.long)

# Convert to PyG Data
edge_index = torch.tensor(list(G.edges())).t().contiguous()
# Make edges bidirectional for undirected graph
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
data = Data(x=features, edge_index=edge_index, y=labels)

# Define train/test masks (train on 4 nodes, test on 2 nodes)
data.train_mask = torch.tensor([1,1,1,1,0,0], dtype=torch.bool)
data.test_mask  = torch.tensor([0,0,0,0,1,1], dtype=torch.bool)

print(data)


Data(x=[6, 6], edge_index=[2, 12], y=[6], train_mask=[6], test_mask=[6])


In [11]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = SimpleGCN()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.01,
    weight_decay=5e-4
)

# Training loop
for epoch in range(200):
    model.train()
    optimizer.zero_grad()

    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])

    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()

print(f'GCN Test Accuracy: {acc:.4f}')


GCN Test Accuracy: 0.8030
